In [1]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt
from ipyparallel import Client
clnts = Client(profile='mpi')
clnts[:].clear()
clnts[:].activate()
%px from mpi4py.MPI import COMM_WORLD as comm

mpiindx = range(len(clnts))
for i in xrange(len(clnts)): mpiindx[clnts[i]['comm.rank']] = i
view = clnts[mpiindx]

mpistackR = lambda x:np.vstack((view[x]))
mpistackD = lambda x: np.sum((view[x]),axis=0)
mpistackP = lambda x: np.concatenate((view[x]),axis=-1)
def mpistackRover(x):
    loc_arrays = []
    for i in xrange(len(view[x])-1):
        dat = view[x][i].copy()
        dat[:,-1] += view[x][i+1][:,0]
        dat = dat[:,1:]
        loc_arrays.append(dat)
    return np.hstack((loc_arrays))

Using matplotlib backend: GTK3Agg


In [2]:
%%px --block --verbose

import numpy as np
from sys import path
path.append('./src/')
path.append('./moduls/')
from species import Specie
from solvers import Solver
from chimera_main import ChimeraRun
import numpy as np
import fimera as chimera

dt = 0.015
BoxGrid     = (-0.01 ,0.01 ,0.08  ,0.0002,0.001  )
BoxGridBeam = (-0.006,0.006,0.0036,0.0001,0.00012)

k_res = 105600.205166
gg = 391.397366/(1.+1.95**2/2)
vb = (1.-gg**-2)**0.5

solver_in = {'MaxAzimuthMode':2, 'Grid':BoxGrid, 'KxShift':k_res,'TimeStep':dt, 'Rcut':0.05, 
'Features':('NoPoissonCorrection',)}

seed_in = {'a0':0.107,'k0':k_res,'x0':0.0,'x_foc':25.0,'Lx':0.00107,'LR':0.00656215} 
solver = Solver(comm,solver_in) 
solver.add_gauss_beam(seed_in)

Devices = (['MagneticUndulator1D',1.95, (1.,0.,1e9,1.),[0.,-0.01,0.01]],)

specie_in = {'RandCell':30, 'Charge':-1., 'Density':56.6, 'Mass':1., 'Grid':BoxGridBeam, 'TimeStep':dt, 
'MomentaMeans':(391.397366,0.,0.), 'MomentaSpreads':(0.03914,0.008,0.008), 'NoiseRemv':(k_res,), 
'Features':(),'Devices':Devices}

specie = Specie(comm,specie_in)

rankstr = str(comm.rank)
while len(rankstr)<3: rankstr = '0'+rankstr
specie.particles = np.load('phs_0_'+rankstr+'_0000000.npy')
specie.particles_cntr = specie.particles[:3]
specie.EB = np.zeros((6,specie.particles.shape[-1]))

#specie.add_particles(specie.gen_parts((-0.00535,0.00535,0.0,0.0034)))
#specie.correct_fel()

MovingFrame = {'TimeStep':dt,'Steps':1,'Velocity':vb}

chimera_in = {'Solvers':(solver,),'Particles':(specie,),'MovingFrames':(MovingFrame,)}

Chimera = ChimeraRun(comm,chimera_in)
Chimera.make_halfstep()

In [3]:
%%px --block --verbose

centr =np.zeros((2,0))
nrg = np.zeros(0)
for i in xrange(4000):
    Chimera.make_step(i*dt)
    #xc = comm.reduce(  (specie.particles[1]*specie.particles[-1]).sum()   )
    #w_div = comm.reduce(specie.particles[-1].sum())
    #yc = comm.reduce(  (specie.particles[2]*specie.particles[-1]).sum()   )
    if comm.rank==0: 
        #centr = np.concatenate((centr, np.array([xc/w_div,yc/w_div])[:,None]),axis=-1)
        if np.mod( i,int(1./dt) )==0:
            nrg = np.r_[ nrg, (abs(solver.EG_fb[:,:,:,:3])**2*solver.Args['EnergyFact']).sum()]
print i

[stdout:0] 3999
[stdout:1] 3999
[stdout:2] 3999


In [4]:
plt.clf()
plt.plot(clnts[mpiindx[0]]['nrg']*2.8e3)

In [72]:
plt.clf()
plt.imshow( clnts[mpiindx[0]]['np.abs(solver.EG_fb[:,:,2,2]).T'],aspect='auto',origin='lower',cmap='spectral')
plt.colorbar()

In [66]:
plt.clf()
plt.plot(clnts[mpiindx[0]]['centr[1]'])

In [76]:
plt.clf()
plt.plot(mpistackP('specie.particles[1]'),mpistackP('specie.particles[2]'),'.',alpha=0.01)

In [ ]:
clnts[mpiindx[0]]['np.abs(solver.EG_fb[:,:,2,2]).T']